In [1]:
import numpy as np import pandas as pd from PIL import Image
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import cv2
import os
print(os.listdir("../input"))
from tqdm import tqdm, tqdm_notebook
# Any results you write to the current directory are saved as output.

['train', 'test', 'train.csv', 'sample_submission.csv']


In [2]:
train_images= os.listdir("../input/train/train/")
train_dir = "../input/train/train/"
test_dir = "../input/test/test/"
test_images= os.listdir("../input/test/test/")
label_df = pd.read_csv('../input/train.csv')
train_labels=label_df['has_cactus']

In [3]:
X_tr = []
Y_tr = []
imges = label_df['id'].values
for img_id in tqdm_notebook(imges):
    X_tr.append(cv2.imread(train_dir + img_id))    
    Y_tr.append(label_df[label_df['id'] == img_id]['has_cactus'].values[0])  
X_tr = np.asarray(X_tr)
X_tr = X_tr.astype('float32')
X_tr /= 255
#Y_tr = np.asarray(Y_tr)

In [4]:
from keras.utils import to_categorical
Y_tr = to_categorical(train_labels, num_classes =2)

Using TensorFlow backend.


In [5]:
X_train.shape

NameError: name 'X_train' is not defined

In [6]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Dropout, Flatten, GlobalAveragePooling2D

resnet_weights_path ='imagenet'
my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
my_new_model.add(Dense(256, activation='relu'))
#my_new_model.add(Dropout(0.5))
my_new_model.add(Dense(2, activation='softmax'))
# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

my_new_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
94658560/94653016 [==============================] - 1s 0us/step


In [7]:
my_new_model.fit(X_tr,Y_tr,validation_split=0.1,epochs=200,batch_size=32,verbose = 2)

Train on 15750 samples, validate on 1750 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/200
 - 15s - loss: 0.1801 - acc: 0.9337 - val_loss: 4.0671 - val_acc: 0.7463
Epoch 2/200
 - 9s - loss: 0.1164 - acc: 0.9546 - val_loss: 1.3623 - val_acc: 0.5994
Epoch 3/200
 - 9s - loss: 0.1114 - acc: 0.9564 - val_loss: 0.1169 - val_acc: 0.9543
Epoch 4/200
 - 9s - loss: 0.0926 - acc: 0.9653 - val_loss: 0.1361 - val_acc: 0.9480
Epoch 5/200
 - 9s - loss: 0.0903 - acc: 0.9648 - val_loss: 0.0902 - val_acc: 0.9629
Epoch 6/200
 - 9s - loss: 0.0877 - acc: 0.9663 - val_loss: 0.0961 - val_acc: 0.9663
Epoch 7/200
 - 9s - loss: 0.0828 - acc: 0.9697 - val_loss: 0.1171 - val_acc: 0.9594
Epoch 8/200
 - 9s - loss: 0.0805 - acc: 0.9703 - val_loss: 0.0966 - val_acc: 0.9617
Epoch 9/200
 - 9s - loss: 0.0789 - acc: 0.9704 - val_loss: 0.1061 - val_acc: 0.9611
Epoch 10/200
 - 9s - loss: 0.0693 - acc: 0.9739 - val_loss: 0.1463 - val_acc: 0.9571
Epoch 11/200
 - 9s - loss: 0.0692 - acc: 0.9733 - val_loss: 0

In [8]:
for layer in my_new_model.layers:
    weights = layer.get_weights() 

In [9]:
my_new_model.save_weights("my_weights.h5")

In [10]:
sub_df = pd.read_csv('../input/sample_submission.csv')

In [11]:
X_ts = []
imges = sub_df['id'].values
for img_id in tqdm_notebook(imges):
    X_ts.append(cv2.imread(test_dir + img_id))    
X_ts = np.asarray(X_ts)
X_ts = X_ts.astype('float32')
X_ts /= 255


In [12]:
X_ts.shape
print(results)

NameError: name 'results' is not defined

In [13]:
results= np.empty((sub_df.shape[0],))
for n in tqdm(range(0,sub_df.shape[0])):
    results[n] = my_new_model.predict(X_ts[n].reshape((1, 32, 32, 3)))[0][1]
    #print (results)
#results = np.argmax(results,axis = 1)

100%|██████████| 4000/4000 [01:25<00:00, 46.71it/s]


In [14]:
f=sub_df['id']
results = pd.DataFrame(results,columns=['has_cactus'])

#results['has_cactus'] = results['has_cactus'].apply(lambda x: 1 if x > 0.75 else 0)
results.head()

,has_cactus
0,1.000000e+00
1,1.000000e+00
2,1.777402e-04
3,3.586429e-11
4,9.937993e-01


In [15]:
results.head()

,has_cactus
0,1.000000e+00
1,1.000000e+00
2,1.777402e-04
3,3.586429e-11
4,9.937993e-01


In [16]:
submission = pd.concat([pd.Series(f,name ='id'),results],axis = 1)

submission.to_csv("samplesubmission.csv",index=False)

In [17]:
submission.head()

,id,has_cactus
0,000940378805c44108d287872b2f04ce.jpg,1.000000e+00
1,0017242f54ececa4512b4d7937d1e21e.jpg,1.000000e+00
2,001ee6d8564003107853118ab87df407.jpg,1.777402e-04
3,002e175c3c1e060769475f52182583d0.jpg,3.586429e-11
4,0036e44a7e8f7218e9bc7bf8137e4943.jpg,9.937993e-01
